In [0]:
#libraties
from pyspark.sql.functions import year, month, dayofmonth


In [0]:
# Get the data frames from Retail Data Analysis notebook using 3 different ways

# Access the global temporary view for customer df
df_customer = spark.sql("SELECT * FROM global_temp.df_customer_GloabalTempBiew")

# Access the  global temporary view for products df
df_products = spark.sql("SELECT * FROM global_temp.df_products_GloabalTempBiew")

# Access the parquet file
df_stores = spark.read.format("parquet").load("/tmp/df_stores_wr")

# Run another notebook
#result = dbutils.notebook.run("Path/to/your/Notebook", timeout_seconds, {"argument": "value"})


In [0]:
print(type(df_customer), type(df_products), type(df_stores))

<class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'> <class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
df_customer.show(3)
df_products.show(3)
df_stores.show(3)

+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
|Customer_ID|  Customer_Name|  Segment|      Country|           City|     State|Postal_Code|Region|Age|
+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
|   CG/12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South| 42|
|   DV/13045|Darrin Van Huff|Corporate|United States|    Los Angeles|California|      90036|  West| 47|
|   SO/20335| Sean O'Donnell| Consumer|United States|Fort Lauderdale|   Florida|      33311| South| 19|
+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
only showing top 3 rows

+---------------+---------------+------------+--------------------+
|     Product_ID|       Category|Sub_Category|        Product_Name|
+---------------+---------------+------------+--------------------+
|FUR-BO-10001798|      Furniture|   Bookcas

In [0]:
#Shape (Rows/Column count) of Source DF
print(" df_customer DF shape: ",(df_customer.count(), len(df_customer.columns)))
print(" df_products DF shape: ",(df_products.count(), len(df_products.columns)))
print(" df_stores DF shape: ",(df_stores.count(), len(df_stores.columns)))


 df_customer DF shape:  (793, 9)
 df_products DF shape:  (1861, 4)
 df_stores DF shape:  (9800, 9)


In [0]:
# Assuming the Date column is of type Date. If it's a string, you might need to convert it first
df_stores=df_stores.withColumn("Order_Year", year("Order_Date"))\
.withColumn("Order_Month", month("Order_Date"))\
.withColumn("Order_Day", dayofmonth("Order_Date"))

In [0]:
df_stores.select(['Order_Date','Order_Year','Order_Month', 'Order_Day']).show(3)

+----------+----------+-----------+---------+
|Order_Date|Order_Year|Order_Month|Order_Day|
+----------+----------+-----------+---------+
|2017-11-08|      2017|         11|        8|
|2017-11-08|      2017|         11|        8|
|2017-06-12|      2017|          6|       12|
+----------+----------+-----------+---------+
only showing top 3 rows



In [0]:
#join custome dataset
df_stores_cust = df_stores.join(df_customer,"Customer_ID", "left").select(df_stores["*"], df_customer["Segment"],df_customer["Region"],df_customer["Age"] )

print((df_stores_cust.count(), len(df_stores_cust.columns)))
#df_stores_cust.columns

(9800, 15)


In [0]:
#Joining Product dataset
df_stores_cust_product = df_stores_cust.join(df_products, "Product_Id").select(df_stores_cust["*"], df_products["Category"],df_products["Sub_Category"])

print((df_stores_cust_product.count(), len(df_stores_cust_product.columns)))
#df_stores_cust_product.columns

(9800, 17)


In [0]:
df_fianl = df_stores_cust_product.withColumnRenamed("Segment", "Cust_Segment")\
    .withColumnRenamed("Region", "Cust_Region")\
    .withColumnRenamed("Age","Cust_Age")\
    .withColumnRenamed("Category", "Product_Cat")\
    .withColumnRenamed("Sub_Category", "Product_SubCat")

df_fianl.columns

#Target Dataset Shape
print("Target Dataset Shape: ",(df_fianl.count(), len(df_fianl.columns)))

Target Dataset Shape:  (9800, 17)


In [0]:
# Write the DataFrame to ADLS as Parquet (or replace with your desired format)
df_fianl.write.mode("overwrite").option("header",True).csv("/mnt/retail/Processed Data/")

In [0]:
#Verify ADLS Written Success!
dbutils.fs.ls("/mnt/retail/Processed Data/")

Out[22]: [FileInfo(path='dbfs:/mnt/retail/Processed Data/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1708298366000),
 FileInfo(path='dbfs:/mnt/retail/Processed Data/_committed_2653544037628221569', name='_committed_2653544037628221569', size=112, modificationTime=1708298363000),
 FileInfo(path='dbfs:/mnt/retail/Processed Data/_started_2653544037628221569', name='_started_2653544037628221569', size=0, modificationTime=1708298359000),
 FileInfo(path='dbfs:/mnt/retail/Processed Data/part-00000-tid-2653544037628221569-d812eb41-5d24-430a-923b-3f9c6e812e63-24-1-c000.csv', name='part-00000-tid-2653544037628221569-d812eb41-5d24-430a-923b-3f9c6e812e63-24-1-c000.csv', size=1429814, modificationTime=1708298362000)]